In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("FinalCoursework") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql import Row

# Helper function to test the correctness of the solutions
def test(var, val, msg=""):
    print("1 test passed.") if var == val else print("1 test failed. " + msg)

In [20]:
# <FILL-IN WITH YOUR CODE>
df_trump = spark.read.option("multiline", True).csv("data/hashtag_donaldtrump.csv", header=True, inferSchema=True)
df_biden = spark.read.option("multiline", True).csv("data/hashtag_joebiden.csv", header=True, inferSchema=True)
# Cache your DataFrame
df_trump.cache()
df_biden.cache()
# <FILL-IN WITH YOUR CODE>

DataFrame[created_at: string, tweet_id: string, tweet: string, likes: string, retweet_count: string, source: string, user_id: string, user_name: string, user_screen_name: string, user_description: string, user_join_date: string, user_followers_count: string, user_location: string, lat: string, long: string, city: string, country: string, continent: string, state: string, state_code: string, collected_at: string]

In [21]:
df_trump.count()

1041276

In [5]:
df_biden.count()

1812041

In [22]:
df_trump.show(5)

+-------------------+--------------------+--------------------+-----+-------------+------------------+--------------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+----------+------------+----------+--------------------+-------------+--------------------+----------+--------------------+
|         created_at|            tweet_id|               tweet|likes|retweet_count|            source|             user_id|           user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|       lat|        long|      city|             country|    continent|               state|state_code|        collected_at|
+-------------------+--------------------+--------------------+-----+-------------+------------------+--------------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+----------+---------

In [23]:
df_biden.show(5)

+-------------------+--------------------+--------------------+-----+-------------+------------------+--------------------+------------------+----------------+--------------------+-------------------+--------------------+--------------------+----------+-------------------+----+--------------------+-------------+----------+----------+--------------------+
|         created_at|            tweet_id|               tweet|likes|retweet_count|            source|             user_id|         user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|       lat|               long|city|             country|    continent|     state|state_code|        collected_at|
+-------------------+--------------------+--------------------+-----+-------------+------------------+--------------------+------------------+----------------+--------------------+-------------------+--------------------+--------------------+----------+-------------------+----+--------